<a href="https://colab.research.google.com/github/rsadaphule/jhu-dnn/blob/main/Group10__7%F0%9F%8F%86%F0%9F%A7%ACGenomics_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🧬Genomics</font>](https://www.kaggle.com/c/10jul23jh-genomics/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
# from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

In [ ]:
# !pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
# !mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
# !cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
# !cp kaggle.json ~/.kaggle/kaggle.json > log       # Alternative location of kaggle.json (without a connection to Google Drive)
# !chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
# !kaggle config set -n competition -v 10jul23jh-genomics # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
# !kaggle competitions download >> log              # download competition dataset as a zip file
# !unzip -o *.zip >> log                            # Kaggle dataset is copied as a single file and needs to be unzipped.
# # !kaggle competitions leaderboard --show           # print public leaderboard

In [ ]:
%%time
%%capture
%reset -f
!pip -q install -U sentence-transformers > log
!pip -q install -U tfds-nightly tensorflow > log
!pip -q install -U sentence-transformers tensorflow_addons > log
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os, plotly, tensorflow_addons as tfa, tensorflow as tf, tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer as SBERT
from keras.layers import Flatten, Dense
from sklearn.svm import SVC, LinearSVC
os.environ['TF_DETERMINISTIC_OPS'] = '1'; os.environ['TF_CUDNN_DETERMINISTIC'] = '1'; # allows seeding RNG on GPU
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60*5): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=10000, precision=4, edgeitems=20, suppress=True)
pd.set_option('display.max_rows', 100, 'display.max_columns', 100, 'display.max_colwidth', 100, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 6.25 s, sys: 778 ms, total: 7.03 s
Wall time: 40.9 s


In [ ]:
vX = pd.read_csv('testX.csv').set_index('id')
tYX = pd.read_csv('trainYX.csv').set_index('id')
vX

,DNA
id,
100000,TTGATTAATAAGATTCCTTGACACCCTTTGTAAAGTTTCTATTTCGTGTGAAATATCTATCTCTTCAAATCCTTTTAATTTATCTAGGTATTTGCT...
100001,ATTAGTAACGGAGGATTTACTAGATGTTTGGATTTATATTCTAATTTTATTCAGGTGGAAGGGATTGTTTTATGATTCAATAGTATACAGAGAATA...
...,...
119998,CGTCGGCATGCTCGGGCAGTGCGGCGGGCCAGCAGCGTGCCAGTTGTCGCGGGGCGGCCGGGCATCGCGGCGCCGGGCGGCAGCACTCCCGCGAAG...
119999,GCGAGGGCACGAAGGCACGACGGCAACGGCGGCGAGGAGCGCTGTGGCAACCGTCTCCGCGTTTGCGTGCGTACAGCCGAGAGCTGGTTCGCGCAG...


In [ ]:
tmr = Timer() # runtime limit (in seconds). Add all of your code after the timer

⏳ started. You have 300 sec. Good luck!


❗Do not modify the setup above.

<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**

Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

  **Student's answer:**

  1. When referencing the post by Ernest Bonat, Ph.D. and Bishes Rayamajhi, M.S. [here](https://medium.com/mlearning-ai/apply-machine-learning-algorithms-for-genomics-data-classification-132972933723), we noticed that k-mer counting was commonly used in bioinformatics and was used in one of their models. K-mers are subsequences of certain specified lengths (k) contained within a DNA sequence. For example, a subsequence of AGAT would have four monomers (A, G, A, and T) three 2-mers (AG, GA, AT), etc. We created frequency counts of how many times these k-mers appeared in the model, and then standardized the counts so that the feature counts would not dominate the model.

  2. To evaluate the effectiveness of the preprocessing elements we were looking at the overall performance of the final model.  We chose to use the accuracy metric for this competition and using these added features we saw an improved performance in both validation accuracy and leaderboard accuracy.

  3. We also tried OneHotEncoding the DNA sequence with limited success (~96% accuracy), so in the end we used the standardized k-mer counts and the encodings produced by SBERT model as our features.

In [ ]:
# K-mer
import pandas as pd
import itertools
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

bases = ['A', 'T', 'G', 'C']

# Generate kmers for k=2, k=3, and k=4
kmers = [''.join(p) for k in range(2, 5) for p in itertools.product(bases, repeat=k)]

# Compute kmer counts for tYX and vX
for kmer in kmers:
    tYX[f'{kmer}_count'] = tYX['DNA'].str.count(kmer)
    vX[f'{kmer}_count'] = vX['DNA'].str.count(kmer)

In [ ]:
# Scaling
# Use a more efficient method to scale the kmer counts
from sklearn.preprocessing import StandardScaler

kmer_cols = [f'{kmer}_count' for kmer in kmers]
scaler = StandardScaler()
kmer_counts = scaler.fit_transform(tYX[kmer_cols])
kmer_counts_val = scaler.transform(vX[kmer_cols])

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

**Student's answer:**

For our model approach, we fed the DNA sequence to [SBERT](https://www.sbert.net) to generate text embedding vectors for each sequence. We chose to 50 for the vector size of the sequences to have uniform vector length, though we also experimented with longer vector lengths. After converting the sequences using embeggings from SBERT and adding our k-mer coutns features, we fed the data to a neural network that had 3 dense layers with batch normalization and dropout after each layer.  The final layer was a dense layer with sigmoid activation.  Layers were initialized with "He - Normal" weights. The model itself was trained using a binary cross entropy loss function and utilized momentum, weight decay, and early stopping to avoid overfitting.
1. From HOML and related course material, we know that initalization of layers can have a large effect on a neural network's ability to learn. Because of this, we experimented with different layer initialization techniques and found the He Normal initialization worked best.

  In general, neural networks with more layers and more data tend to learn better. However, due to limited time constraints in these competitions it is often better to have smaller networks that can be trained quickly and that have quick inference times. We found that a 4 layer network (3 dense + 1 output) layer worked well. To help with regularization and avoid overfitting, we also added dropout and batchnormalization layers in-between the dense layers. The 3 hidden dense layers used the GELU activation.

1. We evaluated the effectiveness of our model by using the validation accuracy with a validation split of 5% of the training data we had available. These validation scores were compared during different model runs to choose the best performing model.

1. In addition to [SBERT](https://www.sbert.net) we also tried other embedding models such as USE as well as other training embeddings.  However, showed worse performance than SBERT using the [all-MiniLM-L12-v2](https://www.sbert.net/docs/pretrained_models.html) embedding. We also tried different vectorized embedding lengths, different number of layers, different layer activation functions, different layer initializations, learning rate schedules, and more. Overall, we found that the chosen model was a good balance between performance and training + inference time.

  Another effort we took was to take a representative subsample of the raw input data before creating the embeddings and the k-mer processing to help keep the notebook runtime down, but it did not improve our overall performance in the end so this method was not employed for the final submission.

**DNN Model Selection**

**Neural Network Model**: Deeper and wider neural network can improve the perforamance overall, but if it is too deep or too wide, it starts to show overfitting. By checking training loss and validation loss graph, we controlled the number of nodes and the number of hidden layer. 3 hidden layers with 256, 512, 256 nodes show the efficient performance within 300 seconds. Also, adding batch normalization which can improve training stability and accelerate convergence is helpful to prevent overfitting. It normalizes the inputs of a layer by subtracting the batch mean and dividing by the batch standard deviation to increase stability.

**Activation Function** is a function that is added into an artificial neural network in order to help the network learn complex patterns in the data. When comparing with a neuron-based model that is in our brains, the activation function is at the end deciding what is to be fired to the next neuron. That is exactly what an activation function does in an ANN as well. It takes in the output signal from the previous cell and converts it into some form that can be taken as input to the next cell. So, it is important to choose a proper activation function. We tested multiple different functions such as ReLU, Swish, Sigmoid,Tahn, LeakyReLU, but Gelu shows the best performance.

**Optimizer** is an algorithm that adapts the neural network's attributes, like learning rate and weights. So, it helps in improving the accuracy and reduces the total loss. We tried multiple optimizer such as SGD, Adagrad, Adadelta, and Rmsprop, but Adam shows the best performance. In Adam, there are multiple parameters to control. Based on experiments, we found the best parameters for better performance.


---


Training parameters such as epoch, learning rate, batch size are important to control model's performance.

**Epoch**: An epoch is a complete pass through the entire training dataset during the training process. In other words, it represents the number of times the model has seen and learned from the entire dataset. Training a model for multiple epochs allows it to refine its performance, but if epoch is too high, it starts to show overfitting. To avoid overfitting, we utilzed earlystopping method which stops epoch before overfitting based on validation loss. It works good to avoid oeverfitting.

**Learning Rate**: The learning rate is a hyperparameter deciding the step size at which the model's parameters are updated during the optimization process. It controls how much the parameters should be adjusted in response to the estimated error. A large learning rate causes the parameters to update too quickly and a small learning rate causes the convergence process to be very slow. So, it is important to use an appropriate learning rate for training within limited time. Based on experiments, we decide to use 0.0076.

**Batch Size**: During model training, the dataset is divided into small batches. The batch size refers to the number of samples that are processed together before the model's parameters are updated. The choice of batch size can have an impact on the model's computational efficiency. Generally, large batch size sometimes decreases model performance, but this binary classification is not difficult analysis. So, we decided to use large batch size to reduce time consuming.

In [ ]:
# SBERT
%%capture
sbert = SBERT('all-MiniLM-L12-v2')
%time tEmb, vEmb = sbert.encode([s[:50] for s in tYX.DNA]), sbert.encode([s[:50] for s in vX.DNA])  # embed all sequences to same-size vectors

In [ ]:
df_tEmb = pd.DataFrame(tEmb)
df_tEmb.reset_index(drop=True, inplace=True)
df_kmer = pd.DataFrame(kmer_counts)
df_kmer.reset_index(drop=True, inplace=True)
tEmb_transformed = pd.concat([df_tEmb, df_kmer], axis=1)
tEmb_transformed.columns = tEmb_transformed.columns.astype(str)

df_vEmb = pd.DataFrame(vEmb)
df_vEmb.reset_index(drop=True, inplace=True)
df_kmer_v = pd.DataFrame(kmer_counts_val)
df_kmer_v.reset_index(drop=True, inplace=True)
vEmb_transformed = pd.concat([df_vEmb, df_kmer_v], axis=1)
vEmb_transformed.columns = vEmb_transformed.columns.astype(str)

In [ ]:
# Normal DNN

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input, Add, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

tf.random.set_seed(0)
#Init = keras.initializers.RandomNormal(seed=0)

kerinit = 'he_normal' #'he_normal'
bs = 256
ep = 25
vs = 0.05
LR = 0.0076

# 256 512 256
m = keras.models.Sequential([
    Flatten(input_shape=[tEmb_transformed.shape[1]], name='input'),
    Dense(256, activation='gelu', kernel_initializer=kerinit, name='hidden1'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(512, activation='gelu', kernel_initializer=kerinit, name='hidden2'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='gelu', kernel_initializer=kerinit, name='hidden3'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid', kernel_initializer=kerinit, name='output')])


early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
m.summary()
m.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=LR,
    beta_1=0.7, #0.7
    beta_2=0.95,
    epsilon=1e-07,
    amsgrad=True,
    weight_decay=0.02, #0.02
    clipnorm=0.5, #0.5
    use_ema=True,
    ema_momentum=0.98), metrics=['accuracy'])

hist = m.fit(tEmb_transformed, np.array(tYX.Y), batch_size=bs, epochs=ep, validation_split=vs, callbacks=[early_stopping])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 720)               0         
                                                                 
 hidden1 (Dense)             (None, 256)               184576    
                                                                 
 batch_normalization_15 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 dropout_15 (Dropout)        (None, 256)               0         
                                                                 
 hidden2 (Dense)             (None, 512)               131584    
                                                                 
 batch_normalization_16 (Ba  (None, 512)               2048      
 tchNormalization)                                    

In [ ]:
pY = pd.DataFrame(m.predict(vEmb_transformed), index=vX.index, columns=['y'])   # predicted targets
ToCSV((pY>0.5)*1, 'MySubmission_Final_v1')

625/625 [==============================] - 2s 2ms/step


# **References:**

- Géron, Aurélien. Hands-on Machine Learning with Scikit-Learn, Keras, and Tensorflow: Concepts, Tools, and Techniques to Build Intelligent Systems, O’Reilly Media, Inc., Sebastopol, CA, 2019.
- Bonat, Ernest and Rayamajhi, Bishes Rayamajhi. Apply Machine Learning Algorithms for Genomics Data Classification, Medium, 2021. https://medium.com/mlearning-ai/apply-machine-learning-algorithms-for-genomics-data-classification-132972933723
- inspectorG4dget. Generating all DNA kmers with Python. StackOverflow, 2014. https://stackoverflow.com/questions/25942528/generating-all-dna-kmers-with-python
- sklearn.preprocessing.StandardScaler. (n.d.). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
- Reimers, Nils. SentenceTransformers Documentation, SBERT.net, 2022. https://www.sbert.net/
- Reimers, Nils. Pretrained Models, SBERT.net, 2022. https://www.sbert.net/docs/pretrained_models.html
- He, Kaiming, et al. Delving Deep into Rectifiers:
Surpassing Human-Level Performance on ImageNet Classification, Computer Vision Foundation, pg. 1026 - 1034
- Yann LeCun, Léon Bottou, Genevieve B. Orr, and Klaus-Robert Müller. 1998. Efficient BackProp. In Neural Networks: Tricks of the Trade, this book is an outgrowth of a 1996 NIPS workshop. Springer-Verlag, Berlin, Heidelberg, 9–50.
- Hendrycks, Dan and Kevin Gimpel. “Gaussian Error Linear Units (GELUs).” arXiv: Learning (2016): n. pag.
- Team, K. (n.d.). Keras documentation: Keras layers API. https://keras.io/api/layers/
- Sergey Ioffe and Christian Szegedy. 2015. Batch normalization: accelerating deep network training by reducing internal covariate shift. In Proceedings of the 32nd International Conference on International Conference on Machine Learning - Volume 37 (ICML'15). JMLR.org, 448–456.
- Team, K. (n.d.-a). Keras documentation: BatchNormalization layer. https://keras.io/api/layers/normalization_layers/batch_normalization/
- "API Reference." Tensorflow, https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping


<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 292 sec


# 💡**Starter Ideas**

1. Learn about DNA [&#127910;](https://www.youtube.com/results?search_query=nucleotides+genes+amino+acids+)
1. Try a larger training sample.
1. Try longer training DNA strings, but SBERT may have a cap on string length, so you might split DNA into several strings and then concatenate or average resulting vectors
1. Try other pretrained SBERT models. Note that DNA sequence uses ACGT letters, but many other models were trained on multilingual text. So, you might prefer those that were trained on mostly ASCII.
1. SBERT is trained on word tokens (typically, separated by spaces), but DNA sequence has no spaces. Try placing spaces after every character or some semantically meaningful subsequences (this might require more domain knowledge).
1. Try Google's [USE](https://tfhub.dev/google/universal-sentence-encoder-multilingual/3) embedding models
1. Try Facebook's [LASER](https://github.com/facebookresearch/LASER) and [others](https://tfhub.dev/s?module-type=text-language-model).
1. Try [Enformer](https://tfhub.dev/deepmind/enformer/1) for gene expressions. See [DeepMind paper](https://deepmind.com/blog/article/enformer).
1. Try building your own embeddings on the given sequences. SBERT and other packages make it easy (just a few lines), but it may take too much time.
1. Assess distribution of character patterns (single, doubles, triplets, ...). For example, an ACGT string generates AC, CG, GT doubles and ACG and CGT triplets. Does one class have more subsequences of some type? This might be a feature in your model.
1. Try features built as counts of subsequences (singles, doubles, triplets, ...). Consider EDA first.
1. Concatenate or otherwise combine multiple embeddings derived from each gene string
1. Learn from [*The genetic code*](https://www.khanacademy.org/science/ap-biology/gene-expression-and-regulation/translation/a/the-genetic-code-discovery-and-properties), Khan Academy.
1. Learn from [*Apply Machine Learning Algorithms for Genomics Data Classification*](https://medium.com/mlearning-ai/apply-machine-learning-algorithms-for-genomics-data-classification-132972933723)
1. Learn from [*Efficient counting of k-mers in DNA sequences using a bloom filter*](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-12-333) Páll Melsted et al. 2011
1. Try [Byte Pair Encoding](https://www.derczynski.com/papers/archive/BPE_Gage.pdf) and [SentencePiece](https://arxiv.org/pdf/1808.06226.pdf) to auto identification of "important" [k-mers](https://en.wikipedia.org/wiki/K-mer) (substrings)